In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import os, shutil
import zipfile

In [ ]:
zip_ref = zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/train.zip', 'r')
zip_ref.extractall('train')
zip_ref.close()

train_full_dir = './train/train/'

create_if_not = ['training/dogs/','training/cats/']
for i in create_if_not:
    if os.path.isdir(i) == 0:
        os.makedirs(i)
train_filenames = os.listdir(train_full_dir)
categories = []
for tr_filename in train_filenames[:8000]:
    category = tr_filename.split('.')[0]
    if category == 'dog':
        shutil.move(train_full_dir+ tr_filename, 'training/dogs/'+ tr_filename)
    else:
        shutil.move(train_full_dir+ tr_filename, 'training/cats/'+ tr_filename)

        
shutil.rmtree('./train')

In [ ]:
width = 180
height = 180
depth = 3
image_size = (width,height)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './training',
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './training',
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal"),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
])

rest_of_the_model = tf.keras.Sequential([
    layers.Conv2D(64, (5,5), activation = 'relu', padding = 'same'),
    layers.MaxPool2D(2, strides = 2),
    layers.Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    layers.Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    layers.MaxPool2D(2, strides = 2),
    layers.Conv2D(256, (3,3), activation = 'relu', padding = 'same'),
    layers.Conv2D(256, (3,3), activation = 'relu', padding = 'same'),
    layers.MaxPool2D(2, strides = 2),
    
    layers.Flatten(),
    layers.Dense(units=512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(units=128, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(units=2, activation='softmax'),
])

In [ ]:
input_shape = (180,180,3)
    
inputs = layers.Input(shape=input_shape)
x = data_augmentation(inputs)
x = keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
outputs = rest_of_the_model(x)

model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=keras.optimizers.SGD(),loss="sparse_categorical_crossentropy",metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_ds, epochs=20, validation_data=val_ds,)

In [ ]:
model.save('t1.h5')

In [ ]:
model.load_weights('./t1.h5')

In [ ]:
history = model.fit(train_ds, epochs=50, validation_data=val_ds,)

In [ ]:
def display_training_curves(training, validation, title, subplot):
    ax = plt.subplot(subplot)
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    ax.set_xlabel('epoch')
    ax.legend(['training', 'validation'])

plt.subplots(figsize=(10,10))
plt.tight_layout()
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 211)
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 212)

In [ ]:
zip_ref = zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/test.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
test_path = './test/'

In [ ]:
import random
test_filenames = os.listdir(test_path)
rand = random.choice(test_filenames)
image_path = test_path + rand
img = keras.preprocessing.image.load_img(image_path, target_size=(180, 180))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
#score = tf.nn.softmax(predictions[0])

print(
"This image most likely belongs to {} class with a {:.2f} percent confidence."
.format(class_names[np.argmax(predictions)].upper(), 100 * np.max(predictions)))

from PIL import Image
Image.open(image_path)  